# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションにおけるエージェントの作成と実行に関するものです。エージェントは、質問を通じてターゲットワードを推測するための戦略を持っています。

### 取り組んでいる問題
このノートブックは、「20の質問」ゲームを模倣した環境で、エージェントがターゲットワードを推測するプロセスをシミュレートすることを目的としています。シンプルなエージェントを複数作成し、それぞれが異なるカテゴリ（電子機器、食べ物、乗り物、国）についての質問を行うことを担います。

### 使用している手法とライブラリ
- **ライブラリ**: 
  - `numpy`: 数値計算に使用
  - `pandas`: データ処理やファイル入出力に使用
  - `kaggle_environments`: Kaggleの環境を作成し、シミュレーションを行うために使用

- **エージェントの構造**:
  - 各エージェントは、質問のターン、推測のターン、および回答のターンに対応する動作を持っています。これにより、ターゲットワードに関連する質問を無作為に選び、関連性に基づいて回答を生成します。

- **動作**: 
  - `simple_agent1`, `simple_agent2`, `simple_agent3`, `simple_agent4`: それぞれ異なるカテゴリに基づいた質問を行い、関連性をチェックすることで、「はい」または「いいえ」で回答します。
  - `is_related`関数を用いて、キーワードが特定のカテゴリに関連しているかどうかを確認します。

- **ゲームの実行**: 
  - `kaggle_environments.make`を使ってゲーム環境を作成し、`env.run`によってエージェントを競わせるシミュレーションを行います。また、最終的にエージェントを提出するために必要なファイル構成を整える手順も含まれています。

このノートブックは、エージェントの開発のための基本的なフレームワークを提供し、シンプルな戦略を用いて「20の質問」ゲームをプレイし、実施結果を確認することができます。

---


# 用語概説 
以下は、提供されたJupyter Notebookの内容に基づいて、初心者がつまずきそうな専門用語の解説です。

1. **エージェント (Agent)**:
   - ゲームやシミュレーションの中で行動をとる主体のこと。ここでは、質問をするエージェント（推測者）と回答するエージェント（回答者）を指します。このノートブックでは、複数のエージェントが競い合う形式で実装されています。

2. **環境 (Environment)**:
   - エージェントが相互作用するシステムのこと。ゲームのルールや条件が定義され、エージェントが入力を行い、出力を受け取ります。ここでは、Kaggleが提供する「llm_20_questions」という名のゲーム環境が使用されています。

3. **obs (Observation)**:
   - エージェントが現在の状態や状況を表すオブジェクト。このオブジェクトには、ターンの種類、過去の質問、過去の回答、推測されたキーワードなどの情報が含まれます。エージェントはこの情報を利用して次に取る行動を決定します。

4. **cfg (Configuration)**:
   - 環境やエージェントの設定情報を含むオブジェクト。具体的には、ゲームの進行に必要な設定やパラメータが格納されていますが、実際のノートブックでは詳細な使用は見られません。

5. **ターン (Turn)**:
   - ゲームの進行状況の単位で、エージェントが質問をしたり、推測をしたりする際の操作。ターンのタイプ（ask, guess, answer）によって、エージェントがどのような行動を取るべきかが決まります。

6. **質問リスト (Questions List)**:
   - エージェントが行う質問の選択肢が含まれたリスト。各エージェントは、自身の役割に合わせた質問をランダムに選択することで、ターゲットを特定しようとします。

7. **代替推測 (Alternative Guesses)**:
   - 提示されたキーワードに対して他の可能性のある候補を提示するリスト。これにより、エージェントは多様な推測を行い、ターゲットの特定を助けます。

8. **Kaggle環境 (Kaggle Environment)**:
   - Kaggle上で動作する競技用のプログラム環境。この特定のノートブックでは、KaggleのAPIを使用して、20の質問を扱うゲームが実装されています。

9. **CSVファイル (CSV file)**:
   - データをカンマ区切り形式で保存したファイル。ここでは、エージェントが候補を保持するために使用され、一般的にデータの入出力に利用されます。

10. **サポートファイル (Support Files)**:
    - 提出に必要な追加ファイルで、例えば、エージェントの動作に必要なデータや構成情報を含むファイルです。このノートブックでは、主にCSVファイルがサポートファイルとして利用されています。

初心者の方がこれらの用語を理解すると、ノートブックの全体像や、エージェントの動作に対する理解が深まるでしょう。

---


In [ ]:
# このPython 3環境には多くの便利な分析ライブラリがインストールされています
# これはkaggle/python Dockerイメージによって定義されています: https://github.com/kaggle/docker-python
# 例えば、ここではいくつかの便利なパッケージを読み込むことができます

import numpy as np # 線形代数用のライブラリ
import pandas as pd # データ処理用のライブラリ, CSVファイルの入出力（例えば、pd.read_csv）

# 入力データファイルは読み取り専用の"../input/"ディレクトリにあります
# 例えば、これを実行すると（実行ボタンをクリックするかShift+Enterを押すことで）、入力ディレクトリ内のすべてのファイルがリスト表示されます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) # 各ファイルのフルパスを表示します

# 現在のディレクトリ（/kaggle/working/）には最大20GBまで書き込むことができ、これは「すべて保存して実行」することで出力として保存されます
# 一時ファイルは/kaggle/temp/に書き込むこともできますが、現在のセッション外では保存されません

In [ ]:
import random

# キーワードがカテゴリーに関連しているかを確認するヘルパー関数
def is_related(keyword, category_keywords):
    return keyword.lower() in category_keywords # キーワードを小文字に変換して、カテゴリーキーワードに含まれているかを確認

# カテゴリーキーワード
electronics_keywords = {"smartphone", "laptop", "tablet", "television"} # 電子機器関連のキーワード
food_keywords = {"pizza", "burger", "sushi", "pasta"} # 食べ物関連のキーワード
vehicles_keywords = {"car", "bike", "plane", "boat"} # 乗り物関連のキーワード
countries_keywords = {"japan", "france", "brazil", "canada"} # 国関連のキーワード

def simple_agent1(obs, cfg):
    electronics_questions = [ # 電子機器についての質問リスト
        "Is it a smartphone?", "Is it a laptop?", 
        "Is it a tablet?", "Is it a television?"
    ]
    
    if obs.turnType == "ask": 
        response = random.choice(electronics_questions) # 質問のターンの時には、ランダムに質問を選択
    elif obs.turnType == "guess": 
        response = "smartphone"  # 推測のターンでは、現在はスマートフォンを推測
    elif obs.turnType == "answer":
        response = "**yes**" if is_related(obs.keyword, electronics_keywords) else "**no**" # 関連性を確認し、回答する
    return response

def simple_agent2(obs, cfg):
    food_questions = [ # 食べ物についての質問リスト
        "Is it a pizza?", "Is it a burger?", 
        "Is it sushi?", "Is it pasta?"
    ]
    
    if obs.turnType == "ask": 
        response = random.choice(food_questions) # 質問のターンの時には、ランダムに質問を選択
    elif obs.turnType == "guess": 
        response = "pizza"  # 推測のターンでは、現在はピザを推測
    elif obs.turnType == "answer":
        response = "**yes**" if is_related(obs.keyword, food_keywords) else "**no**" # 関連性を確認し、回答する
    return response

def simple_agent3(obs, cfg):
    vehicle_questions = [ # 乗り物についての質問リスト
        "Is it a car?", "Is it a bike?", 
        "Is it a plane?", "Is it a boat?"
    ]
    
    if obs.turnType == "ask": 
        response = random.choice(vehicle_questions) # 質問のターンの時には、ランダムに質問を選択
    elif obs.turnType == "guess": 
        response = "car"  # 推測のターンでは、現在は車を推測
    elif obs.turnType == "answer":
        response = "**yes**" if is_related(obs.keyword, vehicles_keywords) else "**no**" # 関連性を確認し、回答する
    return response

def simple_agent4(obs, cfg):
    country_questions = [ # 国に関する質問リスト
        "Is it Japan?", "Is it France?", 
        "Is it Brazil?", "Is it Canada?"
    ]
    
    if obs.turnType == "ask": 
        response = random.choice(country_questions) # 質問のターンの時には、ランダムに質問を選択
    elif obs.turnType == "guess": 
        response = "Japan"  # 推測のターンでは、現在は日本を推測
    elif obs.turnType == "answer":
        response = "**yes**" if is_related(obs.keyword, countries_keywords) else "**no**" # 関連性を確認し、回答する
    return response

In [ ]:
# このセルは空のコードセルです。必要に応じてコードを追加してください。

# ゲーム環境の作成と構成

Kaggle環境は、`make()`関数を使用して作成されます。このとき、*環境*の名前（`"llm_20_questions"`）と、*構成*や*情報*などのいくつかのオプションのデフォルトを指定します。コンペティションと同じようにゲームを実行したい場合は、デフォルトをそのまま使用すれば問題ありません。

In [ ]:
import kaggle_environments

# Kaggle環境を作成するための関数を呼び出します
env = kaggle_environments.make(environment="llm_20_questions")
# 「No pygame installed」というエラーは無視してもかまいません

In [ ]:
print("このセッションのキーワードは: ")
print(kaggle_environments.envs.llm_20_questions.llm_20_questions.keyword) # セッションで使用されるキーワードを表示します
print(" ")
print("一部のキーワードには、代替の推測（alts）のリストもあり、これも受け入れられます。")
print("このセッションの代替のリストは:")
print(kaggle_environments.envs.llm_20_questions.llm_20_questions.alts) # 代替の推測を表示します

# LLM 20 Questionsの実行（デフォルト）

作成した環境（`env`）を使用して、この環境でゲームを実行します。ゲームを実行する際には、4つのエージェントのリストを提出する必要があります：
* "Agent1"（チーム1の推測者）、 
* "Agent2"（チーム1の回答者）、 
* "Agent3"（チーム2の推測者）、 
* "Agent4"（チーム2の回答者）。 

コンペティションでは、推測者か回答者のいずれかを担当するために、無作為にチームメイトとペアになります。

In [ ]:
%%time
# ゲームを実行し、その結果をgame_outputに保存します
# エージェントとして、simple_agent1, simple_agent2, simple_agent3, simple_agent4を使用します
game_output = env.run(agents=[simple_agent1, simple_agent2, simple_agent3, simple_agent4])

In [ ]:
# 環境のレンダリングを行い、ゲームの進行状況を表示します
# 表示モードは"ipython"、幅は600ピクセル、高さは500ピクセルに設定します
env.render(mode="ipython", width=600, height=500)

# 提出可能なエージェントの作成
コンペティションにエージェントを提出するには、エージェントのPythonコードをmain.pyというファイルに書き、サポートファイルと一緒にsubmission.tar.gzにまとめる必要があります。

以下は簡単な例です。もちろん、実際のコンペティションでは、公式のスターターノートブック（https://www.kaggle.com/code/ryanholbrook/llm-20-questions-starter-notebook）にあるような実際のLLMを使用することが望ましいです。ノートブックでLLMエージェントを実行すると、より多くの時間とメモリを要するため、プレイヤー1としてLLMエージェントをテストする場合、プレイヤー2にはシンプルなエージェントを使用することをお勧めします。

In [ ]:
import os

# 提出用の主ディレクトリを指定
submission_directory = "/kaggle/working/submission"
submission_subdirectory = "lib"

# 主ディレクトリが存在しない場合は作成します
if not os.path.exists(submission_directory):
    os.mkdir(submission_directory) # 主ディレクトリを作成
    subdirectory_path = os.path.join(submission_directory, submission_subdirectory) # サブディレクトリのパスを設定
    os.mkdir(subdirectory_path) # サブディレクトリを作成

In [ ]:
import os
import csv

# 主ディレクトリとサブディレクトリを定義します
submission_directory = "/kaggle/working/submission"
submission_subdirectory = "lib"

# 主ディレクトリが存在しない場合は作成します
if not os.path.exists(submission_directory):
    os.mkdir(submission_directory)

# サブディレクトリのフルパスを定義します
subdirectory_path = os.path.join(submission_directory, submission_subdirectory)

# サブディレクトリが存在しない場合は作成します
if not os.path.exists(subdirectory_path):
    os.mkdir(subdirectory_path)

# libディレクトリに例のCSVファイルを作成します
with open(os.path.join(subdirectory_path, "example.csv"), mode='w') as file:
    writer = csv.writer(file)
    writer.writerow(["cow", "horse"]) # CSVファイルに行を書き込みます

In [ ]:
# このセルは空のコードセルです。必要に応じてコードを追加してください。

* エージェントのためのmain.py Pythonコードを記述します
* 環境はmain.pyの最後の関数をエージェントとして使用します。この場合は`agent_fun()`になります。

In [ ]:
%%writefile /kaggle/working/submission/main.py

import os
import sys
import csv
import random

# 他のファイル（例えば、モデルの重み）を提出/libディレクトリに置く場合は、パスを設定する必要があります
KAGGLE_COMPETITION_PATH = "/kaggle_simulations/agent/" # コンペティションパス
if os.path.exists(KAGGLE_COMPETITION_PATH):  # コンペティション内で実行中の場合
    subdirectory_path = os.path.join(KAGGLE_COMPETITION_PATH, "lib")
else: # ノートブック内で実行中の場合
    subdirectory_path = os.path.join("/kaggle/working/submission/", "lib")
sys.path.insert(0, subdirectory_path) # サブディレクトリのパスをシステムパスに追加

# 例のファイルを読み込む
with open(os.path.join(subdirectory_path,"example.csv"), mode='r') as file:
    reader = csv.reader(file)
    guess_list = list(reader) # CSVファイルの内容をリストに変換
    guess_list = guess_list[0] # 最初の行を取得

# 例のファイルからランダムに「動物」を選んでグローバル変数として設定
animal = random.choice(guess_list)
    
# main.py内の最後の関数がエージェント関数となります
def agent_fn(obs, cfg):
    
    # エージェントが推測者で、turnTypeが"ask"の場合
    if obs.turnType == "ask":
        response = f'それは{animal}のように見えますか？' # 質問を作成
    # エージェントが推測者で、turnTypeが"guess"の場合
    elif obs.turnType == "guess":
        if obs.answers[-1] == "yes": # 直前の回答が"yes"の場合
            response = animal # 以前に選んだ動物を推測
        else:
            response = "penguin" # 他の動物として"penguin"を推測
    # エージェントが回答者の場合
    elif obs.turnType == "answer":
        if obs.keyword in obs.questions[-1]: # 直前の質問にキーワードが含まれているか確認
            response = "yes" # 含まれていれば"yes"を返す
        else:
            response = "no" # 含まれていなければ"no"を返す
        
    return response # 最終的な応答を返す

In [ ]:
# このセルは空のコードセルです。必要に応じてコードを追加してください。

この`main.py`ファイルとエージェントは、`/lib/example.csv`サポートファイルと一緒に提出する準備が整いました。

In [ ]:
!apt install pigz pv > /dev/null # pigzとpvをインストールします
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . # submissionディレクトリを圧縮してsubmission.tar.gzを作成します

In [ ]:
# 提出したエージェントを使用してゲームを実行し、その結果をgame_outputに保存します
# 同じmain.pyを使用してエージェントを2つ起動します
game_output = env.run(agents=["/kaggle/working/submission/main.py", "/kaggle/working/submission/main.py", simple_agent3, simple_agent4])
# ゲームの進行状況を表示します
env.render(mode="ipython", width=600, height=500)

In [ ]:
# このセルは空のコードセルです。必要に応じてコードを追加してください。